<a href="https://colab.research.google.com/github/Deepanshu-Paul/Disaster_Tweet/blob/main/Disaster_Tweets_Preprocess_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [112]:
from google.colab import drive
drive.mount('/content/drive')

# Now you can access files in your Drive like regular files.
# Example: Reading a CSV from Drive
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/NLP Practice/Disaster tweets/sample_submission.csv') # Replace with the correct path
#print(df)
train_df = pd.read_csv('/content/drive/MyDrive/NLP Practice/Disaster tweets/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/NLP Practice/Disaster tweets/test.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [113]:
import os
print(os.listdir('/content/drive/MyDrive/NLP Practice/Disaster tweets'))

['test.csv', 'sample_submission.csv', 'train.csv']


In [114]:
## Data Cleaning And Preprocessing
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt_tab') # Download the punkt_tab data for sentence tokenization

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [116]:
corpus="""Hello Welcome,to Krish Naik's NLP Tutorials.
Please do watch the entire course! to become expert in NLP.
"""
#print(corpus)
from nltk.tokenize import sent_tokenize
documents = sent_tokenize(corpus)
#print(documents)

type(documents)

In [118]:
from nltk.tokenize import word_tokenize
words = word_tokenize(corpus)
#print(words)

In [67]:
for sentence in documents:
  print(word_tokenize(sentence))

['Hello', 'Welcome', ',', 'to', 'Krish', 'Naik', "'s", 'NLP', 'Tutorials', '.']
['Please', 'do', 'watch', 'the', 'entire', 'course', '!']
['to', 'become', 'expert', 'in', 'NLP', '.']


In [119]:
from nltk.tokenize import wordpunct_tokenize
#wordpunct_tokenize(corpus)

To learn more about accelerating pandas on Colab, see the [10 minute guide](https://colab.research.google.com/github/rapidsai-community/showcase/blob/main/getting_started_tutorials/cudf_pandas_colab_demo.ipynb) or
 [US stock market data analysis demo](https://colab.research.google.com/github/rapidsai-community/showcase/blob/main/getting_started_tutorials/cudf_pandas_stocks_demo.ipynb).

In [120]:
from nltk.tokenize import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()
#tokenizer.tokenize(corpus)

In [121]:
train_df['text_processed'] = train_df['text'].str.lower()
#train_df.head()

In [122]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
# Download the 'wordnet' dataset
nltk.download('wordnet')

def clean_text_stopwords(text):
  text = re.sub(r'[^a-zA-Z\s]', '', text)
  text = text.lower()
  # Instantiate the WordNetLemmatizer
  lemmatizer = WordNetLemmatizer()
  words =text.split()
  words = [words for words in words if words not in stopwords.words('english')]
  text = ' '.join(words)
   # Lemmatize each word in the list
  words = [lemmatizer.lemmatize(word, pos='v') for word in words]
  text = ' '.join(words)
  words = [lemmatizer.lemmatize(word, pos='n') for word in words]
  text = ' '.join(words)
  words = [lemmatizer.lemmatize(word, pos='a') for word in words]
  text = ' '.join(words)

  return text

train_df['text_processed'] = train_df['text'].apply(clean_text_stopwords)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [123]:
## Create TF-IDF And NGrams
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=2000, ngram_range=(2,3))
tfidf_matrix = vectorizer.fit_transform(train_df['text_processed']).toarray()

In [124]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Create a KMeans instance with the desired number of clusters
# For example, let's use 3 clusters:
kmeans = KMeans(n_clusters=3, random_state=0, n_init='auto')

# Fit the KMeans model to your data (e.g., tfidf_matrix)
kmeans.fit(tfidf_matrix)

# Now you can access the labels
train_df['cluster_kmeans'] = kmeans.labels_


In [75]:
!pip install gensim

In [125]:
def find_optimal_clusters(tfidf_matrix, max_clusters=10):
    """Finds the optimal number of clusters using silhouette score."""
    best_score = -1
    best_n_clusters = 2  # Initialize with a default value
    for n_clusters in range(2, max_clusters + 1):
        kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10) # n_init added for KMeans > 1.0
        kmeans.fit(tfidf_matrix)
        labels = kmeans.labels_
        score = silhouette_score(tfidf_matrix, labels)
        if score > best_score:
            best_score = score
            best_n_clusters = n_clusters
    return best_n_clusters

In [126]:
num_clusters = find_optimal_clusters(tfidf_matrix, max_clusters=10)

In [127]:
kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init=10) # Using provided num_clusters
kmeans.fit(tfidf_matrix)
train_df['cluster_kmeans'] = kmeans.labels_

In [53]:
!pip install --upgrade gensim  # Ensure you have the latest version of Gensim
import gensim
from gensim.models import word2vec # Word2vec is a submodule
#from gensim.models import KeyedVectors
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [128]:
def train_word2vec(sentences, vector_size=3000, window=5, min_count=1, sg=0):  # Added parameters
    """Trains a Word2Vec model."""
    # Use the Word2Vec class to create a new model
    model = gensim.models.Word2Vec(sentences, vector_size=vector_size, window=window, min_count=min_count, sg=sg)
    return model

In [129]:
import numpy as np
def get_average_word_embedding(tokens, model):
  embeddings = []
  for word in tokens:
    if word in model.wv:
      embeddings.append(model.wv[word])
    if embeddings:
      return np.mean(embeddings, axis = 0)
    else:
      return np.zeros(model.vector_size)

In [130]:
sentences = train_df['text_processed'].tolist()
# If training Word2Vec
word2vec_model = train_word2vec(sentences, 3000, 5, 1, 0)
train_df['tweet_embedding'] = train_df['text_processed'].apply(lambda tokens: get_average_word_embedding(tokens, word2vec_model))

In [131]:
train_df['tweet_embedding'] = train_df['text_processed'].apply(lambda tokens: get_average_word_embedding(tokens, word2vec_model))


In [133]:
# Convert embeddings to a matrix for KMeans
embedding_matrix = np.vstack(train_df['tweet_embedding'].values)

kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init=10)
kmeans.fit(embedding_matrix)

KMeans(n_clusters=10, n_init=10, random_state=42)

In [134]:
train_df['wv_cluster'] = kmeans.labels_

In [137]:
# prompt: compare and finalize which is better word2vec or tfidf

# Assuming 'train_df' is your DataFrame with 'cluster_kmeans' and 'wv_cluster' columns
# and the necessary columns for analysis are already created.

def compare_clustering_results(df):
    """Compares the clustering results of TF-IDF and Word2Vec and provides a recommendation."""

    # 1. Compare Cluster Distributions:
    print("Cluster Distribution (TF-IDF):\n", df['cluster_kmeans'].value_counts(normalize=True))
    print("\nCluster Distribution (Word2Vec):\n", df['wv_cluster'].value_counts(normalize=True))

    # 2. Identify Common and Different Cluster Assignments:
    common_assignments = df[df['cluster_kmeans'] == df['wv_cluster']]
    print(f"\nNumber of tweets with common cluster assignments: {len(common_assignments)}")

    different_assignments = df[df['cluster_kmeans'] != df['wv_cluster']]
    print(f"\nNumber of tweets with different cluster assignments: {len(different_assignments)}")

    # 3. Analyze tweets with differing assignments (example - first 5):
    print("\nExamples of tweets with different cluster assignments (first 5):")
    for index, row in different_assignments.head(5).iterrows():
        print(f"Original Text: {row['text'][:100]}...")
        print(f"TF-IDF Cluster: {row['cluster_kmeans']}, Word2Vec Cluster: {row['wv_cluster']}")
        print("-" * 20)

    # 4.  Recommendation based on analysis:
    if len(common_assignments) > len(different_assignments):
        print("\nRecommendation:")
        print("The clustering results are largely similar.  Consider using the method that is more efficient for your purpose.")
        if (df['cluster_kmeans'].value_counts(normalize=True).max() < 0.5 and
                df['wv_cluster'].value_counts(normalize=True).max() < 0.5):
            print("Both methods show reasonable cluster distributions. Consider using TF-IDF due to its simplicity and efficiency")
        else:
            print("TF-IDF is generally faster and simpler. Consider using TF-IDF.")
            print("Word2Vec is potentially more semantically rich but computationally intensive.")

    else:
        print("\nRecommendation:")
        print("The clustering results show substantial differences.")
        print("Further investigation is needed to determine the best approach for your specific task.")
        print("Consider examining the examples with differing assignments to understand the discrepancies.")
        print("If semantic similarity is crucial, Word2Vec might be preferable after further tuning.")

# Call the function to compare results:
compare_clustering_results(train_df)


Cluster Distribution (TF-IDF):
 cluster_kmeans
0    0.964797
4    0.007356
6    0.006436
5    0.004335
1    0.004072
7    0.003678
9    0.003547
3    0.002890
8    0.001970
2    0.000919
Name: proportion, dtype: float64

Cluster Distribution (Word2Vec):
 wv_cluster
1    0.272691
0    0.189413
5    0.121897
4    0.110075
7    0.069224
3    0.062919
8    0.051360
2    0.051228
9    0.048995
6    0.022199
Name: proportion, dtype: float64

Number of tweets with common cluster assignments: 1411

Number of tweets with different cluster assignments: 6202

Examples of tweets with different cluster assignments (first 5):
Original Text: Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all...
TF-IDF Cluster: 0, Word2Vec Cluster: 1
--------------------
Original Text: Forest fire near La Ronge Sask. Canada...
TF-IDF Cluster: 7, Word2Vec Cluster: 9
--------------------
Original Text: All residents asked to 'shelter in place' are being notified by officers. No other evacuation or she

In [139]:
# prompt: run dbscan on tfidf_matrix

from sklearn.cluster import DBSCAN

# Assuming 'tfidf_matrix' is already defined from the provided code

# Create a DBSCAN instance
dbscan = DBSCAN(eps=0.5, min_samples=5) # Adjust eps and min_samples as needed

# Fit the DBSCAN model to the tfidf_matrix
dbscan.fit(tfidf_matrix)

# Get the cluster labels
train_df['cluster_dbscan'] = dbscan.labels_

# Now you can analyze the results, for example:
print(train_df['cluster_dbscan'].value_counts())


cluster_dbscan
 0      4249
-1      1490
 39       37
 47       33
 24       32
        ... 
 140       5
 139       5
 138       5
 73        5
 101       5
Name: count, Length: 204, dtype: int64


In [140]:
# prompt: run dbscan on word2vec matrix

# Assuming 'embedding_matrix' is already defined from the provided code

# Create a DBSCAN instance
dbscan = DBSCAN(eps=0.5, min_samples=5) # Adjust eps and min_samples as needed

# Fit the DBSCAN model to the embedding_matrix
dbscan.fit(embedding_matrix)

# Get the cluster labels
train_df['wv_cluster_dbscan'] = dbscan.labels_

# Now you can analyze the results, for example:
print(train_df['wv_cluster_dbscan'].value_counts())


wv_cluster_dbscan
10    689
11    527
6     479
2     434
17    424
14    424
15    415
9     404
0     401
3     391
7     390
8     388
1     373
12    307
5     273
4     255
13    240
16    173
18    145
20    136
22    120
21    100
19     76
23     22
24     20
25      7
Name: count, dtype: int64


In [141]:
# prompt: compare between the above 2 codes

def compare_clustering_results(df):
    """Compares the clustering results of TF-IDF and Word2Vec and provides a recommendation."""

    # 1. Compare Cluster Distributions:
    print("Cluster Distribution (TF-IDF):\n", df['cluster_kmeans'].value_counts(normalize=True))
    print("\nCluster Distribution (Word2Vec):\n", df['wv_cluster'].value_counts(normalize=True))

    # 2. Identify Common and Different Cluster Assignments:
    common_assignments = df[df['cluster_kmeans'] == df['wv_cluster']]
    print(f"\nNumber of tweets with common cluster assignments: {len(common_assignments)}")

    different_assignments = df[df['cluster_kmeans'] != df['wv_cluster']]
    print(f"\nNumber of tweets with different cluster assignments: {len(different_assignments)}")

    # 3. Analyze tweets with differing assignments (example - first 5):
    print("\nExamples of tweets with different cluster assignments (first 5):")
    for index, row in different_assignments.head(5).iterrows():
        print(f"Original Text: {row['text'][:100]}...")
        print(f"TF-IDF Cluster: {row['cluster_kmeans']}, Word2Vec Cluster: {row['wv_cluster']}")
        print("-" * 20)

    # 4.  Recommendation based on analysis:
    if len(common_assignments) > len(different_assignments):
        print("\nRecommendation:")
        print("The clustering results are largely similar.  Consider using the method that is more efficient for your purpose.")
        if (df['cluster_kmeans'].value_counts(normalize=True).max() < 0.5 and
                df['wv_cluster'].value_counts(normalize=True).max() < 0.5):
            print("Both methods show reasonable cluster distributions. Consider using TF-IDF due to its simplicity and efficiency")
        else:
            print("TF-IDF is generally faster and simpler. Consider using TF-IDF.")
            print("Word2Vec is potentially more semantically rich but computationally intensive.")

    else:
        print("\nRecommendation:")
        print("The clustering results show substantial differences.")
        print("Further investigation is needed to determine the best approach for your specific task.")
        print("Consider examining the examples with differing assignments to understand the discrepancies.")
        print("If semantic similarity is crucial, Word2Vec might be preferable after further tuning.")

# Call the function to compare results:
compare_clustering_results(train_df)


Cluster Distribution (TF-IDF):
 cluster_kmeans
0    0.964797
4    0.007356
6    0.006436
5    0.004335
1    0.004072
7    0.003678
9    0.003547
3    0.002890
8    0.001970
2    0.000919
Name: proportion, dtype: float64

Cluster Distribution (Word2Vec):
 wv_cluster
1    0.272691
0    0.189413
5    0.121897
4    0.110075
7    0.069224
3    0.062919
8    0.051360
2    0.051228
9    0.048995
6    0.022199
Name: proportion, dtype: float64

Number of tweets with common cluster assignments: 1411

Number of tweets with different cluster assignments: 6202

Examples of tweets with different cluster assignments (first 5):
Original Text: Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all...
TF-IDF Cluster: 0, Word2Vec Cluster: 1
--------------------
Original Text: Forest fire near La Ronge Sask. Canada...
TF-IDF Cluster: 7, Word2Vec Cluster: 9
--------------------
Original Text: All residents asked to 'shelter in place' are being notified by officers. No other evacuation or she

# prompt: compare kmeans with word2vec with dbscan with word2vec results

def compare_clustering_results_extended(df):
    """Compares various clustering results and provides a detailed analysis."""

    clustering_methods = {
        'KMeans (TF-IDF)': 'cluster_kmeans',
        'KMeans (Word2Vec)': 'wv_cluster',
        'DBSCAN (TF-IDF)': 'cluster_dbscan',
        'DBSCAN (Word2Vec)': 'wv_cluster_dbscan'
    }

    for method1_name, method1_col in clustering_methods.items():
        for method2_name, method2_col in clustering_methods.items():
            if method1_name != method2_name:
                print(f"\nComparing {method1_name} with {method2_name}:")

                # Cluster Distributions
                print(f"\nCluster Distribution ({method1_name}):\n", df[method1_col].value_counts(normalize=True))
                print(f"\nCluster Distribution ({method2_name}):\n", df[method2_col].value_counts(normalize=True))

                # Common and Different Assignments
                common_assignments = df[df[method1_col] == df[method2_col]]
                print(f"\nNumber of tweets with common cluster assignments: {len(common_assignments)}")

                different_assignments = df[df[method1_col] != df[method2_col]]
                print(f"\nNumber of tweets with different cluster assignments: {len(different_assignments)}")

                # Example Tweets with Different Assignments
                print(f"\nExamples of tweets with different cluster assignments (first 5):")
                for index, row in different_assignments.head(5).iterrows():
                    print(f"Original Text: {row['text'][:100]}...")
                    print(f"{method1_name} Cluster: {row[method1_col]}, {method2_name} Cluster: {row[method2_col]}")
                    print("-" * 20)


# Call the function to compare results:
compare_clustering_results_extended(train_df)


In [ ]:
## Disaster analysis using Naive Bayes
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

